In [256]:
#Importing dependancies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [257]:
#Importing Sklearn
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [258]:
#Loading the cleaned Data
file_path = Path('highered_merged_data.csv')
df = pd.read_csv(file_path)
df.head(5)



c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368


In [259]:
# df_test = df.drop_duplicates()
df_test = df.drop(['FACSTAT', 'ARANK'], axis=1)
df_test.sample(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
152657,2021,170806,4,0,4,Madonna University,Livonia,MI,48150,2,9,2,-83.404722,42.384323
19226,2019,160038,322,128,194,Northwestern State University of Louisiana,Natchitoches,LA,71497-0002,1,9,4,-93.097797,31.750666
26789,2019,177551,1,1,0,Harris-Stowe State University,Saint Louis,MO,63103-2136,1,5,2,-90.224288,38.631744
130042,2021,104090,30,9,21,Brookline College-Phoenix,Phoenix,AZ,85021,3,7,2,-112.112473,33.566102
108496,2020,215105,2,1,1,The University of the Arts,Philadelphia,PA,19102,2,9,2,-75.165888,39.946017
86432,2020,165194,27,15,12,Cape Cod Community College,West Barnstable,MA,2668,1,3,2,-70.337334,41.691911
33001,2019,193016,1,0,1,Mercy College,Dobbs Ferry,NY,10522,2,9,4,-73.874447,41.021626
127257,2020,480967,10,4,6,College of the Muscogee Nation,Okmulgee,OK,74447,1,3,1,-95.936439,35.635288
79286,2020,149222,40,18,22,Southern Illinois University-Carbondale,Carbondale,IL,62901,1,9,4,-89.217273,37.714193
48270,2019,222567,44,18,26,Alvin Community College,Alvin,TX,77511-4898,1,4,3,-95.240671,29.396244


In [260]:
df_test.drop_duplicates(inplace=True)

In [261]:
df_test1 = df_test.groupby(['UNITID', 'Year'])['HRTOTLT'].sum().unstack(fill_value=0)
#.unstack(fill_value=0)
df_test1.head(10)

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
100751,6746,6708,6928
100760,137,127,119
100812,274,256,292
100830,865,1011,1004


In [262]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 100654 to 497286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2019    3756 non-null   int64
 1   2020    3756 non-null   int64
 2   2021    3756 non-null   int64
dtypes: int64(3)
memory usage: 117.4 KB


In [263]:
df_test1.dropna()

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
...,...,...,...
496681,0,0,10
496991,0,0,7
497046,0,0,112


In [264]:
#Using drop() to delete rows based on column value
df_test1.drop(df_test1[df_test1[2019] == 0].index, inplace = True)

In [265]:
#Calculating retention rate
df_test1["Retention_Rate"] = (df_test1[2021]/df_test1[2019] * 100).round(2).astype(float)
df_test1.sample(10)

Year,2019,2020,2021,Retention_Rate
UNITID,,,,
156745,268,259,306,114.18
230807,490,435,399,81.43
107664,340,332,308,90.59
227146,364,349,334,91.76
143233,20,22,22,110.00
243106,332,292,293,88.25
448354,6,6,4,66.67
217615,53,52,52,98.11
219709,1505,1532,1965,130.56


In [266]:
#Adding retention boolean
df_test1["Retention"] = np.where(df_test1['Retention_Rate'] > 99.99, True, False)
df_test1

Year,2019,2020,2021,Retention_Rate,Retention
UNITID,,,,,
100654,874,1176,1027,117.51,True
100663,9874,9961,10208,103.38,True
100690,117,100,120,102.56,True
100706,1578,1514,1534,97.21,False
100724,1026,996,970,94.54,False
...,...,...,...,...,...
494287,7,6,8,114.29,True
494436,13,13,15,115.38,True
494737,19,19,19,100.00,True


In [268]:
#Dropping Retention Rate Column
df_test1.drop(columns=["Retention_Rate"])

Year,2019,2020,2021,Retention
UNITID,,,,
100654,874,1176,1027,True
100663,9874,9961,10208,True
100690,117,100,120,True
100706,1578,1514,1534,False
100724,1026,996,970,False
...,...,...,...,...
494287,7,6,8,True
494436,13,13,15,True
494737,19,19,19,True


In [269]:
#Creating feature
X = df_test1.drop("Retention", axis=1)
X = pd.get_dummies(X)

In [270]:
#Adding a target thought? 
target = ["Retention"]

In [271]:
#Creating a target
y = df_test1.loc[:, target].copy()

In [272]:
#Decribing
X.describe()

Year,2019,2020,2021,Retention_Rate
count,3645.000000,3645.000000,3645.000000,3645.000000
mean,799.189300,776.892730,777.718519,101.006209
std,2164.002758,2098.881278,2145.416521,216.722722
min,1.000000,0.000000,0.000000,0.000000
25%,50.000000,46.000000,44.000000,86.570000
50%,212.000000,201.000000,200.000000,96.740000
75%,601.000000,584.000000,585.000000,105.740000
max,36293.000000,30347.000000,35552.000000,12900.000000


In [273]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train['Retention']))
print(Counter(y_test['Retention']))

Counter({False: 1614, True: 1119})
Counter({False: 526, True: 386})


In [274]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Counter({False: 1614, True: 1119})


In [275]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


1.0

In [276]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[526,   0],
       [  0, 386]], dtype=int64)

In [277]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       1.00      1.00      1.00      1.00      1.00      1.00       526
       True       1.00      1.00      1.00      1.00      1.00      1.00       386

avg / total       1.00      1.00      1.00      1.00      1.00      1.00       912



In [278]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Counter({False: 1614, True: 1119})


In [279]:
# Calculated the balanced accuracy score
y_pred = ee_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


1.0

In [280]:
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred)
cm2

array([[526,   0],
       [  0, 386]], dtype=int64)